In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as actF
import torchvision 
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
  print("using gpu")
else :
  print('using cpu')

using cpu


In [ ]:
epochs = 10
batch_size = 4
learning_rate = 0.001

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


Load the DataSet

In [ ]:
train_data =torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform = transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

dataiter = iter(train_loader)
images, labels = dataiter.next()

Modle Definition

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(actF.relu(self.conv1(x)))  
        x = self.pool(actF.relu(self.conv2(x)))  
        x = x.view(-1, 16 * 5 * 5)            
        x = actF.relu(self.fc1(x))               
        x = actF.relu(self.fc2(x))               
        x = self.fc3(x)                       
        return x

In [ ]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Training the Modle

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/10], Step [2000/12500], Loss: 2.3477
Epoch [1/10], Step [4000/12500], Loss: 1.3782
Epoch [1/10], Step [6000/12500], Loss: 1.5689
Epoch [1/10], Step [8000/12500], Loss: 1.0755
Epoch [1/10], Step [10000/12500], Loss: 0.7717
Epoch [1/10], Step [12000/12500], Loss: 1.0459
Epoch [2/10], Step [2000/12500], Loss: 1.0415
Epoch [2/10], Step [4000/12500], Loss: 1.4459
Epoch [2/10], Step [6000/12500], Loss: 1.0438
Epoch [2/10], Step [8000/12500], Loss: 1.4418
Epoch [2/10], Step [10000/12500], Loss: 0.9714
Epoch [2/10], Step [12000/12500], Loss: 1.2093
Epoch [3/10], Step [2000/12500], Loss: 1.0061
Epoch [3/10], Step [4000/12500], Loss: 0.6327
Epoch [3/10], Step [6000/12500], Loss: 1.8306
Epoch [3/10], Step [8000/12500], Loss: 1.2896
Epoch [3/10], Step [10000/12500], Loss: 0.8759
Epoch [3/10], Step [12000/12500], Loss: 1.1630
Epoch [4/10], Step [2000/12500], Loss: 1.1342
Epoch [4/10], Step [4000/12500], Loss: 1.2619
Epoch [4/10], Step [6000/12500], Loss: 0.6815
Epoch [4/10], Step [8000/125

Testing the Modle

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 61.48 %
Accuracy of plane: 66.0 %
Accuracy of car: 87.54 %
Accuracy of bird: 46.58 %
Accuracy of cat: 34.5 %
Accuracy of deer: 51.84 %
Accuracy of dog: 65.38 %
Accuracy of frog: 69.42 %
Accuracy of horse: 71.18 %
Accuracy of ship: 58.28 %
Accuracy of truck: 64.08 %
